In [1]:
import torch
import os
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.multiprocessing as mp

In [2]:
data_path = os.path.join(os.getcwd(),"Data","iris.csv")
data = pd.read_csv(data_path)
data.columns = ['sepal_length','sepal_width','petal_length','petal_width','class']

In [3]:
Y = data[data.columns[-1]]
X = data[data.columns[:-1]]
X = (X-X.mean())/X.std()

In [4]:
class LinearSVM(nn.Module):
    """Support Vector Machine"""

    def __init__(self):
        super(LinearSVM, self).__init__()
        self.fc = nn.Linear(4, 1)

    def forward(self, x):
        h = self.fc(x)
        return h

In [5]:
def train(X, Y, model, lr=1e-6):
    batchsize = 50
    epoch_num = 10
    X = torch.from_numpy(X.as_matrix()).float()
    Y = torch.from_numpy(Y.as_matrix()).float()
    c=1
    N = len(Y)

    optimizer = optim.SGD(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epoch_num):
        perm = torch.randperm(N)
        sum_loss = 0

        for i in range(0, N, batchsize):
            x = X[perm[i:i + batchsize]]
            y = Y[perm[i:i + batchsize]]

            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            x = Variable(x)
            y = Variable(y)

            optimizer.zero_grad()
            output = model(x)
            loss = torch.mean(torch.clamp(1 - output * y, min=0))  # hinge loss
            loss += c * torch.mean(model.fc.weight**2)  # l2 penalty
            loss.backward()
            optimizer.step()

            sum_loss += loss.data.cpu().numpy()[0]

        if epoch % 1 == 0:
            print('Epoch:{:4d}\tloss:{}'.format(epoch, sum_loss / N))

In [6]:
model = LinearSVM()
num_processes = 4
if torch.cuda.is_available():
    model.cuda()
model.share_memory()
processes = []

for rank in range(num_processes):
    p = mp.Process(target=train, args=(rank, model))
    p.start()
    processes.append(p)
for p in processes:
    p.join()

In [7]:
train(X, Y, model)

Epoch:   0	loss:0.01716463637832027
Epoch:   1	loss:0.017176984940599275
Epoch:   2	loss:0.01669064744206883
Epoch:   3	loss:0.017156691359193534
Epoch:   4	loss:0.017097970383279277
Epoch:   5	loss:0.017129824465553233
Epoch:   6	loss:0.017140000458531732
Epoch:   7	loss:0.017012238902533613
Epoch:   8	loss:0.01722370298116799
Epoch:   9	loss:0.01684927620343714
